# HUHgle Colab Notebook

<img src="https://drive.google.com/uc?export=view&id=1cX3z4BkUDoatQl4d20SGHL8yCkpyF6Iv" width="750" height="525">

This Colab notebook enables easy access to, and use of, HUHgle: a Python-based interactive tool for the visualization, design, and optimization of substrates for HUH-tag mediated covalent labeling of proteins of interest with ssDNA substrates of interest.

HUHgle source code is freely available and can be accessed on Github under an open source license.

**Citing this work**

Any publication that discloses findings and/or figures produced by this notebook should cite the HUHgle paper.

**More information**

You can find more information about HUHgle in its associated paper:

* [HUHgle: An Interactive Substrate Design Tool for Covalent Protein-ssDNA Labeling using HUH-tags](https://www.biorxiv.org/content/10.1101/2024.03.15.585203v1)

You can find more information about HUH-tags in the following papers:

* [Sequence-Directed Covalent Protein–DNA Linkages in a Single Step Using HUH-Tags](https://pubs.acs.org/doi/10.1021/jacs.7b02572)
* [Molecular Underpinnings of ssDNA Specificity by Rep HUH-endonucleases and Implications for HUH-tag Multiplexing and Engineering](https://academic.oup.com/nar/article/49/2/1046/6067397)

You can find more information about the basic science/biology of replication-initiating HUH-endonucleases (HUH-tags in their endogenous context) in the following papers:

* [Breaking and joining single-stranded DNA: the HUH endonuclease superfamily](https://www.nature.com/articles/nrmicro3067)
* [Watson-Crick Base-Pairing Requirements for ssDNA Recognition and Processing in Replication-Initiating HUH Endonucleases](https://journals.asm.org/doi/10.1128/mbio.02587-22)

**Anatomy of an HUH-tag Reaction**

HUH-tags are versatile fusion proteins that mediate sequence-specific protein-ssDNA covalent linkage in a simple, efficient, and cost-effective manner. These fusion partners have emerged as an ideal tool to link proteins of interest to ssDNA sequences of interest because of their minimal requirements for reaction and their long-lived protein-ssDNA bioconjugates.

HUH-tags use an eponymous metal coordinating motif most often comprised of two histidine (H) separated by a bulky hydrophobic residue (U) to coordinate a divalent cation -- either magnesium or manganese -- to their active site which the enzyme uses to polarize the phosphate backbone of a targeted ssDNA substrate, enabling the nuclease's catalytic tyrosine to cleave the nucleic acid and then covalently bind to its newly exposed 5' end.

See the image below for a graphical depiction of this process.

<img src="https://drive.google.com/uc?export=view&id=1uqfqD7zAZFjoo23S_zvNGatXi5ujdoq6" width="975" height="375">

**How to interpret HUHgle plots**

HUHgle plots color each of the four nucleobases differently in order to enhance interpretation -- As, Ts, Gs, and Cs are yellow, orange, green, and red, respectively. Importantly, if HUHgle identifies a sequence with which the user-selected HUH-tag interacts it recolors it to a monochromatic blue gradient indicative of how good of a substrate that sequence is for the HUH-tag. The darker blue a sequence is recolored, the more effective the selected HUH-tag is at interacting with that substrate. See the 'Substrate Score' legend on the right side of the plot. Substrates with scores of 0.8 or higher have their text recolored white for clarity. All HUH-tag substrates are comprised of seven bases upstream of the cleavage site (positions -7 through -1) and two bases downstream of the cleavage site (+1 and +2) -- the covalent bond is formed to the base in position +1. HUHgle identifies cleavage sites by highlighting them with a black vertical line. The base immediately to the right of one of these lines is the +1 position.

In instances where there are two or more sequences that overlap with one another, HUHgle highlights where each contiguous sequence is in the greater sequence context by underlining it with a black horizontal line. When two black horizontal lines partially overlap, as they do across sites two and three in the image below, it indicates the presence of overlapping sequences. The cleavage sites on overlapping sequences are still appropriately indicated. The color indicative of the 'Substrate Score' metric is recolored from the first overlapping base onwards to indicate the efficiency of the second substrate while retaining the color of the underlying first substrate in all positions preceding the overlap.

<img src="https://drive.google.com/uc?export=view&id=1JeWEp5MJazFL_vL6MQU4LDpu6ywf1SnB" width="750" height="420">


**Contact**

If you have any questions or comments, please contact Adam Smiley (adamtreysmiley@gmail.com) and Wendy Gordon (wrgordon@umn.edu)



# HUHgle Search Function

The following cells, labeled 1 - 4, comprise the core functionality of HUHgle. Select your experiment type (i.e., single, two-, or three-way labeling), enter your substrate sequence(s), select your HUH-tag(s), and then search & plot!

*   Progress through the cells in a linear manner (i.e., run through the program cell-by-cell) for the best experience.
*   Carefully read the instructions in each cell and respond to prompts/dropdown menus as they appear.


In [ ]:
#@title 1: Install Required Modules, Load Data, and Declare Functions
#@markdown 1. Run this cell.
#@markdown 2. Wait for the download to complete.
#@markdown  * This should only take a handful of seconds.
#@markdown 3. Proceed to the next cell once the terminal clears and indicates that installation is complete.

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.cm import Blues
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.transforms as transforms
import pandas as pd
import math
import requests
from google.colab import files
from IPython.display import display, clear_output
import zipfile
from itertools import combinations
from statistics import mean
from google.colab import output
from matplotlib.font_manager import FontProperties
output.enable_custom_widget_manager()

# Install required stuff that doesn't come with Colab
!pip install ipympl
!pip install tqdm
!pip install ViennaRNA

##
# Load and Prepare Data
# Direct download link to the file stored on Gordon Lab Google Drive
file_url = "https://drive.google.com/uc?export=download&id=15WmYH0sLUhmwmsiqjjYcQH-EWqe2bjlc"

# Download the file using download link
r = requests.get(file_url, allow_redirects=True)
open('SubstrateScores.csv', 'wb').write(r.content)

# Load the dataset
data = pd.read_csv('SubstrateScores.csv')

# Drop the less useful information -- CpCDV and MSMV are too promiscuous to be
# useful, and WDV(106F) is a catalytically inactive control
data.drop(columns=['CpCDV', 'MSMV', 'WDV(Y106F)'], inplace=True)

# Add 'AC' to the end of each k-mer in the dataset -- interaction sites conclude
# with an 'AC' dinucleotide
data['kmers'] = data['kmers'].apply(lambda x: x + 'AC')

# Function to create a dictionary for a given HUH-tag filtering out sequences with
# substrate scores less than 0.05
def create_dict_from_column(dataframe, column_name):
    return {k: v for k, v in zip(dataframe['kmers'], dataframe[column_name]) if v > 0.05}

# Create a dictionary for each column (except the first two, which are labels)
dictionaries = {col: create_dict_from_column(data, col) for col in data.columns[2:]}

# Global variables used in future functions
sequences = []
current_sequence_index = 0
nucs = list(dictionaries.keys())
selected_option = None

###
# Declare Functions

# Sequence filtering functions
def contains_non_ATGC_characters(sequence_list):
    for sequence in sequence_list:
        for char in sequence:
            if char not in ['A', 'T', 'G', 'C']:
                return True
    return False

def check_long_short_strings(string_list):
    for string in string_list:
        if len(string) > 300:
            return True
        elif len(string) < 9:
            return True
    return False

def has_duplicates(string_list):
    seen_strings = set()

    for string in string_list:
        if string in seen_strings:
            return True
        seen_strings.add(string)

    return False

# Function to recommend three HUH-tags based on the substrate queried in the context of single-labeling
def recommend_huh(substrate, data):

    # Search the substrate for scored sequences and calculate the total percent reduction for each nuclease
    total_reduction = {col: 0 for col in data.columns if col not in ['k-mer #', 'kmers']}
    for _, row in data.iterrows():
        kmer = row['kmers']
        count = substrate.count(kmer)
        for nuclease in total_reduction.keys():
            total_reduction[nuclease] += (row[nuclease] * count)

    # Check if all score values are zero
    if all(value == 0 for value in total_reduction.values()):
        # If all values are zero, return all nuclease options
        print("\nNo preferred HUH-tag -- no predicted cleavage across all candidates -- select any of the eight options.")
        return list(total_reduction.keys())
    elif all(abs(value - mean(total_reduction.values())) / mean(total_reduction.values()) <= 0.05 for value in total_reduction.values()):
        # Check if all values are within 5% of their mean
        print("\nNo preferred HUH-tag -- nearly equivalent predicted cleavage across all candidates -- select any of the eight options.")
        return list(total_reduction.keys())
    else:
        # Otherwise, return the top three keys
        print("\nHUHgle has recommended its top three HUH-tags based on lowest interaction with the specified substrate")
        top_three_keys = sorted(total_reduction, key=total_reduction.get, reverse=False)[:3]
        return top_three_keys

# Logic and functions to score and recommend HUH-tag pairs, triplets, and quadruplets

# Define thresholds

# Scored need to be higher than 0.5
upper_threshold = 0.50
# Scored can't be lower than 0.1
lower_threshold = 0.10

# List of HUH-tags
nucleases = data.columns[2:]

# Function to calculate orthogonality score
def calculate_orthogonality_score(orthogonal_kmers, nucleases):
    scores = {}
    for key, value in orthogonal_kmers.items():
        if not value.empty:
            score = 1
            count_diff = 0
            for nuclease in key:
                if nuclease in value.columns:
                    count = sum(value[nuclease] >= upper_threshold)
                    score *= count
                    count_diff += count
            score -= abs(count_diff)
            scores[key] = score
        else:
            scores[key] = 0
    return scores

# Function to find orthogonal substrates for pairs and triplets
def find_orthogonal_kmers(data, nucleases, num_nucleases):
    orthogonal_kmers_dict = {}
    for nuclease_combination in combinations(nucleases, num_nucleases):
        key = frozenset(nuclease_combination)
        orthogonal_kmers = pd.DataFrame()
        for nuclease in nuclease_combination:
            conditions = [(data[nuclease] >= upper_threshold) if nuclease == n else (data[n] <= lower_threshold) for n in nuclease_combination]
            selected_kmers = data[conditions[0]]
            for condition in conditions[1:]:
                selected_kmers = selected_kmers[condition]
            if not selected_kmers.empty:
                orthogonal_kmers = pd.concat([orthogonal_kmers, selected_kmers]).drop_duplicates()

        orthogonal_kmers_dict[key] = orthogonal_kmers[['kmers'] + list(nuclease_combination)]

    return orthogonal_kmers_dict

# Finding orthogonal k-mers for pairs, triplets, and quadruplets
orthogonal_kmers_pairs = find_orthogonal_kmers(data, nucleases, 2)
orthogonal_kmers_triplets = find_orthogonal_kmers(data, nucleases, 3)
orthogonal_kmers_quadruplets = find_orthogonal_kmers(data, nucleases, 4)

# Scoring orthogonal sequences for pairs and triplets
orthogonality_scores_pairs = calculate_orthogonality_score(orthogonal_kmers_pairs, nucleases)
orthogonality_scores_triplets = calculate_orthogonality_score(orthogonal_kmers_triplets, nucleases)
orthogonality_scores_quadruplets = calculate_orthogonality_score(orthogonal_kmers_quadruplets, nucleases)

# Function to select top scoring sequences from the identified orthogonal sequences
def select_top_scoring(orthogonality_scores, orthogonal_kmers_dict, top_n):
    top_scores = sorted(orthogonality_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return {key: orthogonal_kmers_dict[key] for key, _ in top_scores}

# Selecting top scoring pairs and triplets
top_scoring_pairs = select_top_scoring(orthogonality_scores_pairs, orthogonal_kmers_pairs, 10)
top_scoring_triplets = select_top_scoring(orthogonality_scores_triplets, orthogonal_kmers_triplets, 10)
top_scoring_quadruplets = select_top_scoring(orthogonality_scores_quadruplets, orthogonal_kmers_quadruplets, 3)  # Not in the manuscript or HUHgle, but accessible here

# Function to recommend three HUH-tags based on the substrate queried in the context of single-labeling
def recommend_huh_pair(substrates, data, available_pairs):
    # Initialize total score dictionary for individuals
    total_score_individuals = {col: 0 for col in data.columns if col not in ['k-mer #', 'kmers']}

    # Calculate total score for each individual nuclease
    for substrate in substrates:
        for _, row in data.iterrows():
            kmer = row['kmers']
            count = substrate.count(kmer)
            for nuclease in total_score_individuals.keys():
                total_score_individuals[nuclease] += (row[nuclease] * count)

    # Calculate combined scores for each pair in available pairs
    pair_scores = {}
    for pair in available_pairs:
        nuclease1, nuclease2 = pair.split(', ')
        combined_score = total_score_individuals.get(nuclease1, 0) + total_score_individuals.get(nuclease2, 0)
        pair_scores[pair] = combined_score

    # Check if all scores are zero
    if all(score == 0.0 for score in pair_scores.values()):
        print("\nNo preferred HUH-tag pair -- no predicted cleavage across all candidate pairs -- select any of the options.")
        return available_pairs

    # Check if all scores are within 5% of each other
    if all(abs(score - mean(pair_scores.values())) / mean(pair_scores.values()) <= 0.05 for score in pair_scores.values()):
        print("\nNo preferred HUH-tag pair -- nearly equivalent predicted cleavage across all candidate pairs -- select any of the options.")
        return available_pairs

    # Sort pairs by score and select top three
    top_three_pairs = sorted(pair_scores, key=pair_scores.get)[:3]
    print("\nHUHgle has recommended its top three HUH-tags pairs based on lowest interaction with the specified substrate(s)")
    return top_three_pairs

# Function to recommend three HUH-tags based on the substrate queried in the context of single-labeling
def recommend_huh_triplet(substrates, data, available_triplets):
    # Initialize total reduction dictionary for individuals
    total_score_individuals = {col: 0 for col in data.columns if col not in ['k-mer #', 'kmers']}

    # Calculate total reduction for each individual nuclease
    for substrate in substrates:
        for _, row in data.iterrows():
            kmer = row['kmers']
            count = substrate.count(kmer)
            for nuclease in total_score_individuals.keys():
                total_score_individuals[nuclease] += (row[nuclease] * count)

    # Calculate combined scores for each triplet in available triplets
    triplet_scores = {}
    for triplet in available_triplets:
        nucleases = triplet.split(', ')
        combined_score = sum(total_score_individuals.get(nuclease, 0) for nuclease in nucleases)
        triplet_scores[triplet] = combined_score

    # Check if all scores are zero
    if all(score == 0.0 for score in triplet_scores.values()):
        print("\nNo preferred HUH-tag triplet -- no predicted cleavage across all candidate triplets -- select any of the options.")
        return available_triplets

    # Check if all scores are within 5% of each other
    if all(abs(score - mean(triplet_scores.values())) / mean(triplet_scores.values()) <= 0.05 for score in triplet_scores.values()):
        print("\nNo preferred HUH-tag triplet -- nearly equivalent predicted cleavage across all candidate triplets -- select any of the options.")
        return available_triplets

    # Sort triplets by score and select top three
    top_three_triplets = sorted(triplet_scores, key=triplet_scores.get)[:3]

    print("\nHUHgle has recommended its top three HUH-tags triplets based on lowest interaction with the specified substrate(s)")
    return top_three_triplets

# HUHgle plot functions -- primary and accessory functions

# Primary functions for interactive plot
def recolor_rects_for_sequences(rects, texts, colors, sequence_intensity_dict, ax):
    for rect, text in zip(rects, texts):
        rect.set_facecolor(colors[text.get_text()])
        text.set_color('black')

    sequence_counter = 1

    motif_indices_dict = {}

    for child in ax.get_children():
        if isinstance(child, patches.Rectangle) and (child.get_facecolor() == (0, 0, 0, 1) or child.get_label() == 'underline'):
            child.remove()

    # Color sequences based on intensity using a blue colormap and white text
    for idx, text in enumerate(texts):
        for sequence, intensity in sequence_intensity_dict.items():
            sequence_length = len(sequence)
            if idx + sequence_length <= len(texts):
                current_sequence = ''.join(inner_text.get_text() for inner_text in texts[idx:idx + sequence_length])
                if current_sequence == sequence:
                    # Update the motif indices dictionary
                    sequence_label = f'Seq{sequence_counter}'
                    sequence_counter += 1
                    motif_indices_dict[sequence_label] = [idx, idx + sequence_length - 1]

                    color = Blues(intensity)  # Get the color from the Blues colormap

                    for j, rect in enumerate(rects[idx:idx + sequence_length]):
                        rect.set_facecolor(color)
                        # Change text color to white if intensity is 0.8 or higher
                        if intensity >= 0.8:
                            texts[idx + j].set_color('white')
                        else:
                            texts[idx + j].set_color('black')

                        # Draw a black bar to indicate cleavage site
                        if j == sequence_length - 2:
                            left_edge = rect.get_x()
                            bottom_edge = rect.get_y()
                            height = rect.get_height()
                            black_bar = patches.Rectangle((left_edge, bottom_edge), -0.05, height, color='black')
                            ax.add_patch(black_bar)
                    break

    def check_overlap(seq1, seq2):
        # Overlap occurs if the end index of seq1 is at least the start index of seq2
        return seq1[1] >= seq2[0]

    def add_spacing_values(motif_indices_dict):
        sequence_labels = list(motif_indices_dict.keys())

        for label in sequence_labels:
            motif_indices_dict[label].append(None)

        current_spacing = 0.1
        for i in range(len(sequence_labels) - 1):
            current_label = sequence_labels[i]
            next_label = sequence_labels[i + 1]

            if check_overlap(motif_indices_dict[current_label], motif_indices_dict[next_label]):
                if motif_indices_dict[current_label][2] is None:
                    motif_indices_dict[current_label][2] = current_spacing

                current_spacing = current_spacing + 0.1
                if current_spacing > 0.4:
                    current_spacing = 0.1
                motif_indices_dict[next_label][2] = current_spacing
            else:
                current_spacing = 0.1

    add_spacing_values(motif_indices_dict)

    nts_per_row = 15  # Number of nucleotides per row - 15 looks the nicest in my opinion

    for sequence_label, seq_info in motif_indices_dict.items():
        start_idx, end_idx, underline_spacing = seq_info

        if underline_spacing is None:
            continue

        current_idx = start_idx
        while current_idx <= end_idx:
            segment_start = current_idx
            segment_end = min(end_idx, (current_idx // nts_per_row + 1) * nts_per_row - 1)

            box_width = rects[segment_start].get_width()
            adjustment = 0.05 * box_width

            start_left_edge = rects[segment_start].get_x() + adjustment
            end_right_edge = rects[segment_end].get_x() + rects[segment_end].get_width() - adjustment
            underline_length = end_right_edge - start_left_edge

            bottom_edge = rects[segment_start].get_y()
            underline_height = 0.03
            underline = patches.Rectangle((start_left_edge, bottom_edge + rects[segment_start].get_height() + underline_spacing), underline_length, underline_height, color='black', label='underline')
            ax.add_patch(underline)

            current_idx = segment_end + 1

def onclick(event, rects, texts, ax, colors, sequence_intensity_dict, modified_sequence):
    base_order = ['A', 'T', 'G', 'C']
    for i, (rect, text) in enumerate(zip(rects, texts)):
        if rect.contains(event)[0]:
            current_base = text.get_text()
            next_base = base_order[(base_order.index(current_base) + 1) % len(base_order)]
            text.set_text(next_base)

            # Recolor all rectangles to update sequences
            recolor_rects_for_sequences(rects, texts, colors, sequence_intensity_dict, ax)

            # Update the modified sequence
            modified_sequence[i] = next_base

            ax.figure.canvas.draw()
            break

def plot_dna_sequence(sequence_list, modified_sequence, nts_per_row=15, show_legend=False):
    global current_sequence_index, virus_name, temp
    sequence_str = sequence_list[current_sequence_index]

    row_height = 1.6
    fig_width = (nts_per_row + 3)
    fig_height = row_height * (math.ceil(len(sequence_str) / nts_per_row) + 1)
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    colors = {
        'A': '#F2C14E',
        'T': '#F78154',
        'G': '#5FAD56',
        'C': '#B4436C'
    }
    number_bg_color = '#D3D3D3'

    rects = []
    texts = []

    x = 1.5
    y = 0
    width = 1
    height = 1
    current_row = 1

    font_size = 24

    for idx, base in enumerate(sequence_str):
        color = colors[base]

        rect = patches.Rectangle((x, y), width, height, facecolor=color)
        ax.add_patch(rect)
        text = ax.text(x + 0.5, y + 0.5, base, ha='center', va='center', color='black', weight='bold', fontsize=font_size)

        rects.append(rect)
        texts.append(text)

        if (idx + 1) % nts_per_row == 0 or (idx + 1) == len(sequence_str):
            start_number = (current_row - 1) * nts_per_row + 1
            ax.add_patch(patches.Rectangle((0, y), 1.5, height, facecolor=number_bg_color))
            ax.add_patch(patches.Rectangle((x + 1, y), 1.5, height, facecolor=number_bg_color))
            ax.text(0.75, y + 0.5, str(start_number), va='center', ha='center', color='black', fontsize=18, weight='bold')
            ax.text(x + 1.75, y + 0.5, str(idx + 1), va='center', ha='center', color='black', fontsize=18, weight='bold')

            y += row_height
            x = 1.5
            current_row += 1
        else:
            x += width

    # Initially recolor all rectangles to update sequences
    recolor_rects_for_sequences(rects, texts, colors, sequence_intensity_dict, ax)

    ax.set_xlim(0, nts_per_row + 3)
    num_rows = len(sequence_str) // nts_per_row
    if len(sequence_str) % nts_per_row > 0:
        num_rows += 1

    ax.set_ylim(0, row_height * num_rows)
    ax.axis('off')
    plt.gca().invert_yaxis()

    legend_font = FontProperties(weight='bold', size=24)

    if show_legend:
        legend_elements = [patches.Patch(facecolor=color, edgecolor='none', label=base) for base, color in colors.items()]
        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.0), ncol=len(colors), edgecolor='white', prop=legend_font)

    # Find the virus name from the sequence_intensity_dict
    virus_name = [key for key, value in dictionaries.items() if value == sequence_intensity_dict][0]

    # Set the title for the plot using the virus name
    if len(sequence_list) == 1:
        ax.set_title(f'{virus_name} Cleavage Profile', fontsize=36, weight='bold', pad=30)
    else:
        ax.set_title(f'{virus_name} Cleavage Profile - Substrate {current_sequence_index + 1}', fontsize=36, weight='bold', pad=30)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.5)
    norm = Normalize(vmin=0, vmax=1)
    cb = ColorbarBase(cax, cmap=Blues, norm=norm, orientation='vertical')

    if len(sequence_str) <= 15:
        cb.set_label('Substrate\nScore', fontsize=28, weight='bold')
    else:
        cb.set_label('\nSubstrate Score', fontsize=28, weight='bold')

    # Set tick labels to bold
    for label in cb.ax.yaxis.get_ticklabels():
        label.set_weight('bold')
        label.set_fontsize(18)

    # Define modified_sequence to track changes
    modified_sequence = list(sequence_str)
    temp = modified_sequence

    # Connect the onclick event and pass modified_sequence
    fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, rects, texts, ax, colors, sequence_intensity_dict, modified_sequence))

    plt.tight_layout()
    plt.gcf().set_dpi(75)
    plt.show()

# Accessory functions for cleavage site and reaction product information

def calculate_dna_molecular_weight(dna_sequence):
    # Average molecular weights of the nucleotides in g/mol
    weights = {
        'A': 313.21,
        'T': 304.2,
        'C': 289.18,
        'G': 329.21
    }

    # Calculate the total weight of the nucleotides
    nucleotide_weight = (sum(weights[nucleotide] for nucleotide in dna_sequence))

    # Convert g/mol to kDa (1 g/mol = 1 Da, and 1000 Da = 1 kDa)
    molecular_weight_kDa = round(nucleotide_weight / 1000, 2)

    return molecular_weight_kDa

def find_primary_products(sequence, sequences_of_interest):
# Primary products are those produces from an HUH-tag interacting with a substrate
    results = []
    for seq in sequences_of_interest:
        index = sequence.find(seq)
        while index != -1:
            # Append the part of the sequence from the match to the end
            results.append(sequence[index+7:])
            # Find next occurrence by searching in the remaining sequence
            index = sequence.find(seq, index + 1)
    return results

def find_secondary_products(results, sequences_of_interest):
# Secondary products are those produces from an HUH-tag interacting with a substrate
# that is already covalently bound to an HUH-tag, reducing the size of the initial conjugate
# this is rare to see in a bioconjugation reaction with 1-to-1 enzyme-substrate ratios
    extracted_sequences = []
    for result in results:
        for seq in sequences_of_interest:
            index = result.find(seq)
            if index != -1:
                # Extract the part of the substring from the beginning up to index 6 of the sequence of interest
                extracted_sequences.append(result[:index+7])
    return extracted_sequences

clear_output()
print("Installation complete - proceed to the next cell.")

In [ ]:
#@title 2: Select Experiment Type and Input ssDNA Substrate Sequence(s) to Query
#@markdown 1. Run this cell. (More specificity in what will happen when you click run)
#@markdown 2. Select experiment type from the dropdown menu (Single, Two-, or Three-Way Labeling).
#@markdown 3. Enter ssDNA substrate sequence(s) to search (sequences must be between 9 and 300 nt in length).
#@markdown  * HUHgle can include an optional base color key at the bottom of the plots it produces.
#@markdown  * You can either add your own HUH-motif at the 5' end of your ssDNA substrate(s), or HUHgle can append one to your substrate(s) based on your selected/recommended HUH-tag(s).
#@markdown  * It is **highly recommended that you use HUHgle to append HUH-motifs to your substrates in orthogonal labeling reactions** because there are so few combinations that enable non-cross reactive, unique cleavage events.
#@markdown  * See the schematic below for a graphical representation of the different experiment options:

#@markdown <img src="https://drive.google.com/uc?export=view&id=1H-idaK-VOwBF3BU5JvjPxU0Bi-GPMa2B" width="1665" height="300">

#@markdown 4. Hit the "Submit Sequence(s)" button and move on to the next cell.

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

import ipywidgets as widgets
from IPython.display import display, clear_output

print("Welcome to HUHgle Colab!\n\n1. Select 'Experiment Type' from the dropdown menu below.\n\n    - See the reaction schematics above for more details on possible experiment options.\n\n    - For two- and three-way labeling, enter the desired number of substrate(s) and leave undesired sequence fields blank.\n\n2. Enter the ssDNA substrate sequence(s) to search ⬇️\n\n3. Hit the 'Submit Sequence(s)' button and proceed to the next cell.\n")

# HTML Widget for the Dropdown Label
dropdown_label = widgets.HTML(
    value="<b>Experiment Type:</b>",
    layout=widgets.Layout(margin='0 0 5px 0')
)

# Dropdown for Experiment Type
experiment_type_dropdown = widgets.Dropdown(
    options=[('', 0), ('Single-Labeling', 1), ('Two-Way Labeling', 2), ('Three-Way Labeling', 3)],
    value=0,
    description='',
)

# Text Input Widgets for Labeling
text_inputs = [widgets.Text(description=f'Sequence {i+1}:') for i in range(3)]

# Container for Text Inputs and Checkbox
input_container = widgets.VBox()

# Global variable to store the legend display state
global_legend = False

# Function to update the global variable when the checkbox is toggled
def update_legend_state(change):
    global global_legend
    global_legend = change.new

# Legend Toggle Checkbox -- New Addition
legend_toggle = widgets.Checkbox(
    value=False,
    disabled=False,
    indent=False
)

# Attach the observer to the legend toggle checkbox
legend_toggle.observe(update_legend_state, names='value')

# Label for the legend toggle checkbox
legend_description = widgets.Label(
    value="Click this box to include a base color key on the HUHgle plot.",
    layout=widgets.Layout(margin='0 0 0 -285px')  # Adjust margin as needed
)

# Arrange the Checkbox and Label in a Horizontal Box
legend_with_description = widgets.HBox([legend_toggle, legend_description])

# Global variable for the checkbox state
checkbox_state = False

# Checkbox Widget
checkbox_widget = widgets.Checkbox(
    value=False,
    disabled=False,
    indent=False
)

# Label Widget for the checkbox
checkbox_description = widgets.Label(
    value="Click this box if you want HUHgle to append an HUH-motif to the 5' end of your substrate.",
    layout=widgets.Layout(margin='0 0 0 -285px')
)

# Arrange the Checkbox and Label in a Horizontal Box
checkbox_with_description = widgets.HBox([checkbox_widget, checkbox_description])

# Global variable for multi-labeling
two_labeling = False
three_labeling = False

def update_checkbox_state(change):
    global checkbox_state
    checkbox_state = change.new

# Observe checkbox for changes and update checkbox_state accordingly
checkbox_widget.observe(update_checkbox_state, 'value')

submit_button = widgets.Button(description="Submit Sequence(s)", layout=widgets.Layout(margin='10px 0 0 0'))

# Update Input Fields Based on Experiment Type -- Updated Section
def update_input_fields(*args):
    global two_labeling, three_labeling

    number_of_inputs = experiment_type_dropdown.value
    elements_to_display = [legend_with_description]
    if number_of_inputs > 0:
        elements_to_display.append(checkbox_with_description)
    elements_to_display.extend(text_inputs[:number_of_inputs])

    # Button Widget inside this function
    if number_of_inputs > 0:

        # Bind the callback function to the button
        submit_button.on_click(on_submit_clicked)

        # Add the submit button to the elements to display
        elements_to_display.append(submit_button)

    input_container.children = elements_to_display

    # Set multi_labeling based on the dropdown selection
    two_labeling = number_of_inputs == 2  # True for Two-Way False otherwise
    three_labeling = number_of_inputs == 3  # True for Three-Way Labeling, False otherwise

experiment_type_dropdown.observe(update_input_fields, 'value')

# New Output Container for Messages
message_output = widgets.Output()

# Callback Function for the Button
def on_submit_clicked(b):
    global sequences
    with message_output:
        clear_output(wait=True)
        sequences = []
        entered_sequences = [input_field.value for input_field in text_inputs if input_field.value.strip() != '']
        if not entered_sequences:
            print("\nPlease enter at least one substrate sequence before pressing the 'Submit' button.")
        else:
            sequences = [sequence.replace(" ", "").upper() for sequence in entered_sequences]
            if contains_non_ATGC_characters(sequences):
                print("\nNon-nucleotide characters detected, please re-enter your sequence(s) and press the 'Submit' button.")
                sequences = []
            elif check_long_short_strings(sequences):
                print("\nSubstrate sequence(s) of inappropriate length detected, please re-enter your sequence and press the 'Submit' button.\n\nSee the instructions above for more details on substrate length restrictions.")
                sequences = []
            elif has_duplicates(sequences):
                print("\nOne or more of your entered substrate sequences are identical -- please re-enter your sequence and press the 'Submit' button.\n\nSee the instructions above for more details on options in orthogonal labeling experiments.")
                sequences = []
            else:
                print("\nValid Sequence(s) entered - Please proceed to next cell.")

        for input_field in text_inputs:
            input_field.value = ''

        if sequences:
            output = "\nEntered Sequence(s):\n"
            for i, sequence in enumerate(sequences, start=1):
                output += f"Sequence {i}: {sequence}\n"
            print(output)

# Bind the callback function to the button
submit_button.on_click(on_submit_clicked)

# Display the HTML Label, Dropdown Menu, Input Container, Button, and Message Output
display(dropdown_label)
display(experiment_type_dropdown)
display(input_container)
display(message_output)

In [ ]:
#@title 3: Select HUH-tag(s) for Labeling

#@markdown 1. Run this cell.
#@markdown 2. Respond to prompts using the drop-down menus below.
#@markdown  * You can either select from the complete list of HUH-tags, or HUHgle can recommend a subset of HUH-tags based on minimizing predicted interaction with the queried substrate sequence(s).
#@markdown 3. Proceed to the next cell after making a final selection.

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

import ipywidgets as widgets
from IPython.display import display, clear_output

# Function to make naming in ortho pair dictionaries more clear
def convert_frozensets_to_names(frozenset_list):
    return [', '.join(nuclease for nuclease in fs) for fs in frozenset_list]

def convert_names_to_frozenset(name_string):
    nuclease_list = name_string.split(', ')
    return frozenset(nuclease_list)

def convert_string_to_list(name_string):
    return name_string.split(', ')

# Function to create a follow-up dropdown with its label
def create_followup_dropdown():
    followup_label_html = widgets.HTML(
        value="<b style='min-width: 300px; display: inline-block;'>Select HUH-tag(s):</b>"
    )
    followup_dropdown = widgets.Dropdown(
        options= nucs,
        value=None,
        disabled=False
    )
    followup_dropdown.observe(on_followup_dropdown_change, names='value')
    return followup_label_html, followup_dropdown

# Function to handle changes in the first dropdown
def on_first_dropdown_change(change):
    global nucs
    nucs = list(dictionaries.keys())
    if change['type'] == 'change' and change['name'] == 'value' and change['new']:
        if change['new'] == 'Yes, recommend HUH-tag.':
            # Follow up for Single - Recommend
            nucs = recommend_huh(sequences[0], data)
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]
        elif change['new'] == 'Select my own HUH-tag.':
            # Follow up for Single - Select own
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]
        elif change['new'] == 'Yes, recommend HUH-tag pairs.':
            # Follow up for Two-Way - Recommend
            nucs = recommend_huh_pair(sequences, data, convert_frozensets_to_names(list(top_scoring_pairs.keys())))
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]
        elif change['new'] == 'Select my own HUH-tag pair.':
            # Follow up for Two-Way - Select own
            nucs = convert_frozensets_to_names(list(top_scoring_pairs.keys()))
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]
        elif change['new'] == 'Yes, recommend HUH-tag triplets.':
            # Follow up for Three-Way - Recommend
            nucs = recommend_huh_triplet(sequences, data, convert_frozensets_to_names(list(top_scoring_triplets.keys())))
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]
        elif change['new'] == 'Select my own HUH-tag triplet.':
            # Follow up for Three-Way - Select own
            nucs = convert_frozensets_to_names(list(top_scoring_triplets.keys()))
            followup_label, followup_dropdown = create_followup_dropdown()
            vbox.children = [first_label_html, first_dropdown, followup_label, followup_dropdown]

def on_followup_dropdown_change(change):
    global selected_option
    if change['type'] == 'change' and change['name'] == 'value' and change['new']:
        if two_labeling == False and three_labeling == False:
            selected_option = [change['new']]
            print("\nHUH-tag(s) Selected - Proceed to Next Cell")
        else:
            selected_option = convert_string_to_list(change['new'])
            print("\nHUH-tag(s) Selected - Proceed to Next Cell")

# HTML widget for the label of the first dropdown
first_label_html = widgets.HTML(
    value="<b style='min-width: 300px; display: inline-block;'>Recommend HUH-tag(s)?:</b>"
)

if len(sequences) == 0:
    clear_output(wait=True)
    print("No Sequence Entered - Please Return to Cell 2")

elif two_labeling == False and three_labeling == False:
    first_dropdown = widgets.Dropdown(
        options=['', 'Yes, recommend HUH-tag.', 'Select my own HUH-tag.'],
        value=None,
        disabled=False
    )
    first_dropdown.observe(on_first_dropdown_change)

    # VBox to contain the dropdowns, labels, and button
    vbox = widgets.VBox(children=[first_label_html, first_dropdown])

    # Display the VBox
    display(vbox)

elif two_labeling == True:
    first_dropdown = widgets.Dropdown(
        options=['', 'Yes, recommend HUH-tag pairs.', 'Select my own HUH-tag pair.'],
        value=None,
        disabled=False
    )
    first_dropdown.observe(on_first_dropdown_change)

    # VBox to contain the dropdowns, labels, and button
    vbox = widgets.VBox(children=[first_label_html, first_dropdown])

    # Display the VBox
    display(vbox)

elif three_labeling == True:
    first_dropdown = widgets.Dropdown(
        options=['', 'Yes, recommend HUH-tag triplets.', 'Select my own HUH-tag triplet.'],
        value=None,
        disabled=False
    )
    first_dropdown.observe(on_first_dropdown_change)

    # VBox to contain the dropdowns, labels, and button
    vbox = widgets.VBox(children=[first_label_html, first_dropdown])

    # Display the VBox
    display(vbox)

In [ ]:
#@title 4: Search and Plot!
#@markdown 1. Run this cell to generate the HUHgle plot.
#@markdown  * Make modifications to the sequence by directly interacting with the plot. Cycle through nucleotides via clicking and the plot will dynamically recalculate HUH-tag interaction sites and their cleavage efficiencies.
#@markdown  * Press the "Reset" button to re-plot your original sequence.
#@markdown  * Use the "Next Substrate" and "Next HUH-tag" buttons (when available) to cycle through your entered substrates or selected HUH-tags, respectively.
#@markdown 2. Click on the "Download Plot & Info" button to download the plot image and associated product information. This information will also print to the terminal below the plot.

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

import ipywidgets as widgets
from IPython.display import display, clear_output

%matplotlib widget

# HUH-tag reactions are generally more efficient if their motif isn't at the
# immediate 5' end of a sequence and if there's a second 'C' after the final
# 'AC' dinucleotide -- this is commonly found in endogenous contexts

# The stuffer gives some padding on the 5' end and the padder adds the 'C' base
# to the 3' end to avoid any potential issues

stuffer = "GCTCT"
padder = "C"
universal_motif = "TAATATTACC"

if len(sequences) == 0:
    print("No Sequence(s) Entered -- Please Return to Cell 2")

elif selected_option is None:
    print("No HUH-tag(s) Selected -- Please Return to Cell 3")

else:

    ###

    sequence_list = [''] * len(sequences)

    cycle_nucs = selected_option

    current_HUH_index = 0

    current_sequence_index = 0

    sequence_intensity_dict = dictionaries[cycle_nucs[current_HUH_index]]

    ###

    def find_max_kmer_for_column(dataframe, column_name):
        max_index = dataframe[column_name].idxmax()
        max_kmer = dataframe.loc[max_index, 'kmers']
        return max_kmer

    def find_optimal_kmer(dataframe, column_names, column_name):
        best_score = float('-inf')
        best_kmer = None

        for index, row in dataframe.iterrows():
            current_kmer = row['kmers']
            current_reduction = row[column_name]

            # Calculate the impact on other nucleases
            impact_score = sum(row[name] for name in column_names if name != column_name)

            # Scoring strategy (maximize current_reduction and minimize impact)
            combined_score = current_reduction - impact_score

            # Update best k-mer if a better score is found
            if combined_score > best_score:
                best_score = combined_score
                best_kmer = current_kmer

        return best_kmer

    # Handle appending HUH-motifs to sequences
    if checkbox_state == True:
        if len(sequence_list) == 1:
            if len(selected_option) == 1:
                sequence_list[0] = stuffer + universal_motif + sequences[0] + padder
            elif len(selected_option) == 2:
                sequence_list[0] = stuffer + find_optimal_kmer(orthogonal_kmers_pairs[frozenset(selected_option)], selected_option, selected_option[0]) + sequences[0] + padder
            elif len(selected_option) == 3:
                sequence_list[0] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[0]) + sequences[0] + padder

        elif len(sequence_list) == 2:
            if len(selected_option) == 2:
                sequence_list[0] = stuffer + find_optimal_kmer(orthogonal_kmers_pairs[frozenset(selected_option)], selected_option, selected_option[0]) + sequences[0] + padder
                sequence_list[1] = stuffer + find_optimal_kmer(orthogonal_kmers_pairs[frozenset(selected_option)], selected_option, selected_option[1]) + sequences[1] + padder
            if len(selected_option) == 3:
                sequence_list[0] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[0]) + sequences[0] + padder
                sequence_list[1] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[1]) + sequences[1] + padder

        elif len(sequence_list) == 3:
            sequence_list[0] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[0]) + sequences[0] + padder
            sequence_list[1] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[1]) + sequences[1] + padder
            sequence_list[2] = stuffer + find_optimal_kmer(orthogonal_kmers_triplets[frozenset(selected_option)], selected_option, selected_option[2]) + sequences[2] + padder
    else:
        sequence_list = sequences

    ###

    button0 = widgets.Button(description="Reset")
    button1 = widgets.Button(description="Download Plot & Info")
    button2 = widgets.Button(description="Next Substrate")
    button3 = widgets.Button(description="Next HUH-tag")


    if two_labeling == False and three_labeling == False:
        hbox = widgets.HBox([button0, button1])
    elif two_labeling == True:
        if len(sequence_list) >1:
            hbox = widgets.HBox([button0, button1, button2, button3])
        else:
            hbox = widgets.HBox([button0, button1, button3])
    elif three_labeling == True:
        if len(sequence_list) >1:
            hbox = widgets.HBox([button0, button1, button2, button3])
        else:
            hbox = widgets.HBox([button0, button1, button3])

    display(hbox)

    def on_button0_click(b):
        with output:
            clear_output(wait=True)
            plot_dna_sequence(sequence_list, sequence_intensity_dict, show_legend=global_legend)

    def on_button1_click(b):
        global virus_name, temp
        plt.savefig(f'HUHglePlot_{virus_name}_CleavageProfileSubstrate{current_sequence_index + 1}.png', dpi=300)
        final_sequence = ''.join(temp)

        with open(f'Output_{virus_name}_CleavageProfileSubstrate{current_sequence_index + 1}.txt', 'w') as file:
            pass

        def custom_print(*args, **kwargs):
            # Print to the terminal
            print(*args, **kwargs)
            # Print to the file
            with open(f'Output_{virus_name}_CleavageProfileSubstrate{current_sequence_index + 1}.txt', 'a') as file:
                print(*args, **kwargs, file=file)

        custom_print(f'Selected Nuclease: {virus_name}\n')

        custom_print("Final Modified Sequence:", final_sequence, "\n")

        present_sequences = {sequence: intensity for sequence, intensity in sequence_intensity_dict.items() if sequence in final_sequence}

        # Sort the motifs by their first occurrence in the sequence
        sorted_motifs = sorted(present_sequences.items(), key=lambda x: final_sequence.find(x[0]))

        # Store the start and end locations of each motif
        motif_locations = {}

        if sorted_motifs:
            custom_print("Cleaved Motifs in the Modified Sequence:")
            for index, (sequence, intensity) in enumerate(sorted_motifs, start=1):
                # Find all locations of the sequence in the final sequence
                locations = [range(i+1, i+len(sequence)+1) for i in range(len(final_sequence)) if final_sequence.startswith(sequence, i)]
                motif_locations[index] = locations
                locations_str = ', '.join([f"{min(loc)}-{max(loc)}" for loc in locations])  # Convert list to comma-separated string
                custom_print(f"Motif {index}: {sequence}, Substrate Score: {intensity * 1:.2f}, Location: nucleotides {locations_str}")
        else:
            custom_print("Cleaved Motifs in the Substrate: None")

        sequences_of_interest = list(sequence_intensity_dict.keys())

        primary_products = find_primary_products(final_sequence, sequences_of_interest)

        secondary_products = find_secondary_products(primary_products, sequences_of_interest)

        # Print primary and secondary products
        custom_print("\nPrimary Products of the Modified Sequence:")
        if primary_products:
            # Dictionary to associate motifs with their primary products
            motif_to_products = {index: [] for index in motif_locations}

            for product in primary_products:
                # Determine which motif produced this product by checking the endpoints
                product_start = final_sequence.find(product) + 1
                product_end = product_start + len(product) - 1
                product_range = range(product_start, product_end + 1)

                for index, locations in motif_locations.items():
                    if any(product_range.start in loc or product_range.stop in loc for loc in locations):
                        motif_to_products[index].append(product)
                        break

            # Report products in the order of the motifs
            for index in sorted(motif_to_products):
                for product in motif_to_products[index]:
                    custom_print(f"From Motif {index}: {virus_name}-{product} (+ {calculate_dna_molecular_weight(product)} kDa)")

        else:
            custom_print("None")

        custom_print("\nSecondary Products of the Modified Sequence:")
        if secondary_products:
            for product in secondary_products:
                custom_print(f"{virus_name}-{product} (+ {calculate_dna_molecular_weight(product)} kDa)")
        else:
            custom_print("None")

        # Create a ZIP file
        with zipfile.ZipFile(f'BatchedFiles_{virus_name}_Substrate{current_sequence_index + 1}.zip', 'w') as zipf:
            # Add files
            zipf.write(f'HUHglePlot_{virus_name}_CleavageProfileSubstrate{current_sequence_index + 1}.png')
            zipf.write(f'Output_{virus_name}_CleavageProfileSubstrate{current_sequence_index + 1}.txt')

        # Download the ZIP file
        files.download(f'BatchedFiles_{virus_name}_Substrate{current_sequence_index + 1}.zip')

        print("\nPlot and Cleavage Information Download Complete\n")

    def on_button2_click(b):
        global current_sequence_index
        with output:
            clear_output(wait=True)
            current_sequence_index = (current_sequence_index + 1) % len(sequence_list)
            plot_dna_sequence(sequence_list, sequence_intensity_dict, show_legend=global_legend)

    def on_button3_click(b):
        global current_HUH_index, sequence_intensity_dict
        with output:
            clear_output(wait=True)
            current_HUH_index = (current_HUH_index + 1) % len(cycle_nucs)
            sequence_intensity_dict = dictionaries[cycle_nucs[current_HUH_index]]
            plot_dna_sequence(sequence_list, sequence_intensity_dict, show_legend=global_legend)

    button0.on_click(on_button0_click)
    button1.on_click(on_button1_click)
    button2.on_click(on_button2_click)
    button3.on_click(on_button3_click)

    output = widgets.Output()

    with output:
        plot_dna_sequence(sequence_list, sequence_intensity_dict, show_legend=global_legend)

    display(output)

# HUHgle Bonus Features

The following cells contain functions that we found to be useful (but not always essential) in the design and optimization of HUH-tag substrates.

In [ ]:
#@title Bonus Feature #1: 'Expanded' Mode
#@markdown 1. Run this cell.
#@markdown 2. Toggle the button displayed below on or off to expand the searching capabilities of HUHgle via *in silico* prediction.
#@markdown  * Replication-initiating HUH-endonuclease have both sequence- and strucural requirements for ssDNA substrate recognition and interaction. We have observed that the identity of the -4 and +1 positions in a cleavage substrate can be highly variable, but they must be able to form a proper Watson-Crick (WC) base pairing for cleavage to occur (see *anatomy of an HUH-tag reaction* above). Expanded mode broadens the scope of HUHgle by predicting cleavage efficiencies for substrates containing non-cognate WC base pairings between the -4 and +1 positions. Importantly, there are exceptions to this rule and some non-WC combinations can enable cleavage (depending on the HUH-tag).
#@markdown 3. **All main and bonus HUHgle functions will use expanded mode when this is switched on.**

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

import ipywidgets as widgets
from IPython.display import display

# Function to expand the dictionaries with new sequences
def expand_dictionaries(dicts):
    for col, kmers_dict in dicts.items():
        new_entries = {}
        for kmer, value in kmers_dict.items():
            if kmer[3] == "T":
                # Create new sequences with replacements at index 3 and 7
                new_kmers = [
                    kmer[:3] + "A" + kmer[4:7] + "T" + kmer[8:],
                    kmer[:3] + "G" + kmer[4:7] + "C" + kmer[8:],
                    kmer[:3] + "C" + kmer[4:7] + "G" + kmer[8:]
                ]
                for new_kmer in new_kmers:
                    new_entries[new_kmer] = value  # Assign parent sequence value to new sequences
        # Update the dictionary with new entries
        kmers_dict.update(new_entries)

# Create a dictionary for each column
dictionaries = {col: create_dict_from_column(data, col) for col in data.columns[2:]}

def on_toggle_button_click(change):
    global dictionaries
    if change['new']:  # Check if the button is pressed
        toggle_button.description = 'On'
        expand_dictionaries(dictionaries)
    else:
        toggle_button.description = 'Off'
        dictionaries = {col: create_dict_from_column(data, col) for col in data.columns[2:]}

toggle_button = widgets.ToggleButton(
    value=False,
    description='Off',
    disabled=False,
    button_style='',
    tooltip=''
)

print("\nToggle Expanded Mode:\n")

# Attach the event handler to the button
toggle_button.observe(on_toggle_button_click, names='value')

display(toggle_button)


In [ ]:
#@title Bonus Feature #2: Cleavage Calculations on Substrates Containing Ambiguous Nucleotides
#@markdown 1. Run this cell.
#@markdown 2. Enter ssDNA substrate sequence(s) to search (sequences must be between 9 and 300 nt in length).
#@markdown  * Please note that you **should not include** an HUH-motif in this kind of analysis because it will complicate interpretation of the results.
#@markdown 3. Hit the 'Submit Sequence' button to finalize your substrate submission.
#@markdown 4. Select an HUH-tag from the available candidates in the dropdown menu.
#@markdown  * Upon selection, substrate analysis will begin and associated files will automatically print to the terminal and download.

#@markdown Information regarding restrictions placed on ambiguous nucleotide-containing substrates:
#@markdown * Any of the nucleotides outlined in the table below are suitable for use in substrates in this HUHgle bonus feature.
#@markdown * We have limited the number of ambiguous nucleotides allowed in a substrate sequence to **eight** nucleotides. Calculations with substrates containing more than eight of these nucleotides are not suitable for the Colab environment.
#@markdown    * See HUHgle's GitHub page for access to the source code in order to support a greater number of ambiguous nucleotides, if necessary.

#@markdown This table shows the nucleotide ambiguity codes and their corresponding designations:

#@markdown | Code | Designation           |
#@markdown |------|------------------------|
#@markdown | M    | A or C                 |
#@markdown | V    | A, C, or G             |
#@markdown | R    | A or G                 |
#@markdown | H    | A, C, or T             |
#@markdown | W    | A or T                 |
#@markdown | D    | A, G, or T             |
#@markdown | S    | C or G                 |
#@markdown | B    | C, G, or T             |
#@markdown | Y    | C or T                 |
#@markdown | N    | A, C, G, or T          |
#@markdown | K    | G or T                 |

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

%matplotlib widget

from itertools import product
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import files
import zipfile
import os

iub_ambiguity_codes = {
    "M": ["A", "C"],
    "V": ["A", "C", "G"],
    "R": ["A", "G"],
    "H": ["A", "C", "T"],
    "W": ["A", "T"],
    "D": ["A", "G", "T"],
    "S": ["C", "G"],
    "B": ["C", "G", "T"],
    "Y": ["C", "T"],
    "N": ["A", "C", "G", "T"],
    "K": ["G", "T"]
}

# Global variable definition
HUH_choice = None
sequence = ""

###

def generate_sequences(sequence):
    # Replace each ambiguous nucleotide with the corresponding list from the dictionary
    lists_to_product = [iub_ambiguity_codes[nucleotide] if nucleotide in iub_ambiguity_codes else [nucleotide] for nucleotide in sequence]

    # Use itertools.product to generate all combinations
    return [''.join(seq) for seq in product(*lists_to_product)]

def contains_invalid_characters(sequence_list, iub_ambiguity_codes):
    # Combine ATGC with the keys from the IUB ambiguity codes dictionary
    valid_chars = set(['A', 'T', 'G', 'C']) | set(iub_ambiguity_codes.keys())

    # Initialize a dictionary to hold counts for each sequence
    ambiguous_char_count = {sequence: 0 for sequence in sequence_list}

    for sequence in sequence_list:
        for char in sequence:
            if char in iub_ambiguity_codes:  # Check if the character is in the IUB codes
                ambiguous_char_count[sequence] += 1
            elif char not in valid_chars:
                print("\nNon-nucleotide characters detected, please re-enter your sequence and press the 'Submit' button.\n")
                return True

    if ambiguous_char_count[sequence] >= 9:
        print("\nToo many ambiguous nucleotide characters detected, please re-enter your sequence and press the 'Submit' button.\n")
        return True

    return False

def analyze_sequences_by_huh_tag(huh_tag, sequences, huh_tag_data):
    if huh_tag not in huh_tag_data:
        print("HUH-tag data not found.")
        return

    # Bins for scores
    high_reduction, moderate_reduction, low_reduction = [], [], []
    all_reductions = []
    all_motifs_and_reductions = []

    # Access the specific dictionary for the given HUH-tag
    specific_huh_tag_data = huh_tag_data[huh_tag]

    for sequence in tqdm(sequences, desc="Analyzing sequences"):
        for motif, reduction in specific_huh_tag_data.items():
            if motif in sequence:
                all_reductions.append(reduction)
                all_motifs_and_reductions.append([motif, reduction])
                if 1 >= reduction >= 0.75:
                    high_reduction.append(reduction)
                elif 0.75 > reduction >= 0.35:
                    moderate_reduction.append(reduction)
                elif 0.35 > reduction >= 0.05:
                    low_reduction.append(reduction)

    # Prepare output string
    print("\n")
    output = (
        f"Total Number of Substrates Analyzed: {len(sequences)}\n"
        f"High Scoring Motifs: {len(high_reduction)}\n"
        f"Moderate Scoring Motifs: {len(moderate_reduction)}\n"
        f"Low Scoring Motifs: {len(low_reduction)}\n"
        f"\nTotal Number of Predicted Cleavage Sites: {len(all_reductions)}\n"
        f"\nKEY:\nCutoff for High Scoring: 100 - 75%\n"
        f"Cutoff for Moderate Scoring: 75 - 35%\n"
        f"Cutoff for Low Scoring: 35 - 5%\n\n"
    )

    # Print the output
    print(output)

    # Save output to file
    text_file = f"analysis_{huh_tag}.txt"
    with open(text_file, 'w') as file:
        file.write(output)

    with open(text_file, 'a') as file:
        for i in all_motifs_and_reductions:
            file.write(f"Motif: {i[0]}, Substrate Score: {i[1]}\n")

    # Plotting
    plot_file = None
    if len(all_reductions) != 0:
        plt.figure(figsize=(10, 6))
        plt.hist(all_reductions, bins=50, color='#6666FF', edgecolor='None', alpha=0.5)
        plt.title(f'Substrate Score Histogram for {huh_tag}', fontsize=28)
        plt.xlabel('Substrate Score', fontsize=20)
        plt.ylabel('Count', fontsize=20)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.xlim(0.0, 1.0)
        plt.grid(alpha=0.75)

        # Save plot to file
        plot_file = f"histogram_{huh_tag}.png"
        plt.savefig(plot_file)

        # Display plot
        plt.show()

    else:
        print("No cleavage detected -- plotting canceled.")

    # Create a zip file containing the output text file and plot
    zip_file = f"analysis_{huh_tag}.zip"
    with zipfile.ZipFile(zip_file, 'w') as zipf:
        zipf.write(text_file, os.path.basename(text_file))
        if plot_file:
            zipf.write(plot_file, os.path.basename(plot_file))

    # Download the zip file
    files.download(zip_file)

###

print("\nEnter the ssDNA substrate sequence to search ⬇️\n")

# Text Input Widget for the Sequence
single_sequence_input = widgets.Text(description='Sequence:')

# HTML Widget for the Dropdown Label
dropdown_label_html = widgets.HTML(
    value="<b>Select HUH-tag:</b>",
    layout=widgets.Layout(margin='0 0 5px 0')
)

# Dropdown for HUH-tag selection without the built-in description
huh_tag_dropdown = widgets.Dropdown(
    options=list(dictionaries.keys()),  # Replace with actual options
    value=None,
    disabled=True
)

# HTML Widget for the HUH-motif Confirmation Dropdown Label
huh_motif_label_html = widgets.HTML(
    value="<b>Does this sequence contain an intentional HUH-motif that lacks ambiguous characters?</b>",
    layout=widgets.Layout(margin='10px 0 5px 0')
)

# New Dropdown for HUH-motif Confirmation
huh_motif_confirmation_dropdown = widgets.Dropdown(
    options=['', 'Yes', 'No'],
    description='',
    disabled=True
)

def on_huh_motif_yes_selected():
    # slider widget
    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(sequence[0]),
        step=1,
        description='',
        continuous_update=True
    )

    # Output widget for displaying the updated sequence
    sequence_output = widgets.Output()

    # Submit button
    submit_button = widgets.Button(description="Submit Sequence")

    # Callback function to update the sequence based on the slider value
    def update_sequence(change):
        global trimmed_sequence
        with sequence_output:
            sequence_output.clear_output(wait=True)
            trimmed_sequence = sequence[0][change['new']:]
            print("Updated Sequence:", trimmed_sequence, "\n")

    # Callback function for the submit button
    def on_submit_clicked(b):
        global trimmed_sequence
        # Perform the analysis with the modified sequence
        print("\n")
        temp = generate_sequences(trimmed_sequence)
        analyze_sequences_by_huh_tag(HUH_choice, temp, dictionaries)

    # Attach the callback functions to the slider and button
    slider.observe(update_sequence, names='value')
    submit_button.on_click(on_submit_clicked)

    # Initially display the full sequence
    with sequence_output:
        print("Updated Sequence:", sequence[0], "\n")

    # Display the slider and the output widget
    display(widgets.VBox([slider, sequence_output, submit_button]))

# Callback Function for the HUH-motif Confirmation Dropdown
def on_huh_motif_confirmation(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'Yes':
            print("\nTrim the HUH-motif away from the rest of your sequence using the slider below:\n\n    - Slide right to remove bases from the 5' end of your sequence.\n\n    - The presence of HUH-motifs within ambiguous nucleotide-containing sequences (like barcoding oligos) complicates subsequent analyses.\n\n    - See the supporting information from the HUHgle manuscript for more details on this.\n")
            on_huh_motif_yes_selected()  # Call the function to display the slider and output widget
            global trimmed_sequence
            trimmed_sequence = sequence[0]
        elif change['new'] == 'No':
            print("\n")
            temp = generate_sequences(sequence[0])
            analyze_sequences_by_huh_tag(HUH_choice, temp, dictionaries)

# Attach the callback function to the HUH-motif confirmation dropdown
huh_motif_confirmation_dropdown.observe(on_huh_motif_confirmation, names='value')

# Modified Callback Function for the HUH-tag Dropdown
def on_huh_tag_selection(change):
    global HUH_choice
    if change['type'] == 'change' and change['name'] == 'value':
        HUH_choice = change['new']
        huh_motif_confirmation_dropdown.disabled = False
        display(widgets.VBox([huh_motif_label_html, huh_motif_confirmation_dropdown]))

# Attach the modified callback function to the huh_tag_dropdown
huh_tag_dropdown.observe(on_huh_tag_selection, names='value')

# VBox to arrange the label above the dropdown
huh_tag_dropdown_container = widgets.VBox([dropdown_label_html, huh_tag_dropdown])

# Button Widget for Submission
single_submit_button = widgets.Button(description="Submit Sequence", layout=widgets.Layout(margin='10px 0 0 0'))

# New Output Container for Messages
single_message_output = widgets.Output()

# Callback Function for the Button
def on_single_submit_clicked(b):
    global sequence
    with single_message_output:
        clear_output(wait=True)
        sequence = [single_sequence_input.value.replace(" ", "").upper()]
        if sequence[0] == '':
            print("\nPlease enter a substrate sequence before pressing the 'Submit' button.")
        elif check_long_short_strings(sequence):
            print("\nSubstrate sequence of inappropriate length detected, please re-enter your sequence and press the 'Submit' button.\n\nSee the instructions above for more details on substrate length restrictions.")
        elif contains_invalid_characters(sequence, iub_ambiguity_codes):
            print("See text above for information regarding constraints placed on sequences containing ambiguous nucleotide characters.")
        else:
            print(f"\nValid Sequence entered: {sequence[0]}\n")
            huh_tag_dropdown.disabled = False
            display(huh_tag_dropdown_container)

        # Clear the text in the input field
        single_sequence_input.value = ''  # This line clears the input field

# Bind the callback function to the button
single_submit_button.on_click(on_single_submit_clicked)

# Display the Input Widget, Checkbox Container, Button, and Message Output
display(single_sequence_input)
display(single_submit_button)
display(single_message_output)


In [ ]:
#@title Bonus Feature #3: HUHgle.FASTA
#@markdown 1. Run this cell.
#@markdown 2. Press the 'Upload File' button to select and upload your .FASTA file containing your DNA sequence of interest.
#@markdown    * While substrate sequences can be of any length, they can not contain ambiguous nucleotides.
#@markdown 4. Interact with subsequent dropdown menus.
#@markdown    * Select an HUH-tag.
#@markdown    * Select 'yes' if your file represents a circular sequence so HUHgle can identify sites that span the end-start junction.
#@markdown    * Select 'yes' if your file is a viral genome from phylum *Cressdnaviricota* and HUHgle will predict its origin of replication.
#@markdown 6. Press the 'submit' button to run analysis.

###

if 'data' not in locals() and 'data' not in globals():
    raise NameError("PLEASE RUN CELL 1 BEFORE ATTEMPTING TO RUN OTHER CELLS!")

###

print("Select and upload a .FASTA file by pressing the 'Choose Files' button below ⬇️\n")

from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import RNA
import re

###

def sanitize_filename(filename):
    # Remove any characters that are not alphanumeric, underscore, or dot
    sanitized_filename = re.sub(r'[^\w\-.]', '_', filename)
    return sanitized_filename

def read_fasta(file_path):
    with open(file_path, 'r') as file:
        # Initialize variables to hold the ID and sequence
        sequence_id = None
        sequence_data = []

        for line in file:
            line = line.strip()
            if line.startswith(">"):  # Check if it's the ID line
                if sequence_id is not None:
                    break
                sequence_id = line[1:]  # Remove '>' and save the ID
                sequence_id = sanitize_filename(sequence_id)  # Sanitize the sequence ID
            else:
                sequence_data.append(line)  # Add sequence line to the list

        return sequence_id, ''.join(sequence_data)  # Join the sequence parts into a single string
###

# Function to validate the file extension
def is_fasta_file(filename):
    return filename.lower().endswith('.fasta')

# Function to create the dropdown menus and the submit button
def create_interaction_elements():
    # HTML widgets for labels
    label_html_huh_tag = widgets.HTML(
        value="<b>Select HUH-tag:</b>",
        layout=widgets.Layout(margin='10px 0 5px 0')
    )
    label_html1 = widgets.HTML(
        value="<b>Is this a circular sequence?</b>",
        layout=widgets.Layout(margin='10px 0 5px 0')
    )
    label_html2 = widgets.HTML(
        value="<b>Is the sequence from phylum Cressdnaviricata?</b>",
        layout=widgets.Layout(margin='10px 0 5px 0')
    )

    # Dropdown menus
    dropdown_menu_huh_tag = widgets.Dropdown(
        options=list(dictionaries.keys()),
        description='',
        disabled=False,
    )
    dropdown_menu1 = widgets.Dropdown(
        options=['No', 'Yes'],
        description='',
        disabled=False,
    )
    dropdown_menu2 = widgets.Dropdown(
        options=['No', 'Yes'],
        description='',
        disabled=False,
    )

    # Button Widget for Submission
    submit_button = widgets.Button(description="Submit", layout=widgets.Layout(margin='10px 0 0 0'))

    # Display the labels, dropdown menus, and the submit button
    display(widgets.VBox([label_html_huh_tag, dropdown_menu_huh_tag, label_html1, dropdown_menu1, label_html2, dropdown_menu2, submit_button]))

    # Callback function for the submit button
    def on_submit_clicked(b):
        clear_output()
        print("Selections:")
        print(f"Circular sequence: {'True' if dropdown_menu1.value == 'Yes' else 'False'}")
        print(f"Sequence from Cressdnaviricota: {'True' if dropdown_menu2.value == 'Yes' else 'False'}")

        # Check if the sequence is circular and adjust accordingly
        if dropdown_menu1.value == 'Yes':
            sequence_to_search = fasta_file[1][-30:] + fasta_file[1]  # Add the last 30 bp to the front
            circular = True
        else:
            sequence_to_search = fasta_file[1]  # Use the sequence as is for linear DNA
            circular = False

        if dropdown_menu2.value == 'Yes':
            cress_bool = True
        else:
            cress_bool = False

        if contains_non_ATGC_characters(fasta_file[1]) is True:
            print("\nNon-nucleotide characters detected, please re-run this cell and upload a .FASTA file that contains only As, Ts, Gs, or Cs.")

        else:
            sequence_intensity_dict = dictionaries[dropdown_menu_huh_tag.value]

            with open(f'Output_{dropdown_menu_huh_tag.value}_CleavageProfileSubstrate_GB:{fasta_file[0]}.txt', 'w') as file:
                pass

            def custom_print(*args, **kwargs):
                # Print to the file
                with open(f'Output_{dropdown_menu_huh_tag.value}_CleavageProfileSubstrate_GB:{fasta_file[0]}.txt', 'a') as file:
                    print(*args, **kwargs, file=file)

            custom_print(f'Selected Nuclease: {dropdown_menu_huh_tag.value}\n')

            present_sequences = {sequence: intensity for sequence, intensity in sequence_intensity_dict.items() if sequence in sequence_to_search}

            # Sort the motifs by their first occurrence in the sequence
            sorted_motifs = sorted(present_sequences.items(), key=lambda x: sequence_to_search.find(x[0]))

            # Adjust locations for motifs found in the appended section for circular sequences
            adjusted_motif_locations = []
            fold_motif_information = []

            for motif, intensity in sorted_motifs:
                motif_locations = [i for i in range(len(sequence_to_search)) if sequence_to_search.startswith(motif, i)]

                for location in motif_locations:
                    if circular and location < 30:
                        # Motif starts within the appended section, adjust position to reflect original sequence
                        adjusted_location = [len(fasta_file[1]) - 30 + location, location - 30 + len(motif)]
                        adjusted_motif_locations.append([adjusted_location, motif, intensity])
                        fold_motif_information.append([[location, location + len(motif)], motif, intensity])
                    elif circular and location >= 30:
                        adjusted_location = [location - 30, location - 30 + len(motif)]
                        adjusted_motif_locations.append([adjusted_location, motif, intensity])
                        fold_motif_information.append([[location, location + len(motif)], motif, intensity])
                    elif not circular:
                        adjusted_location = [location, location + len(motif)]
                        adjusted_motif_locations.append([adjusted_location, motif, intensity])
                        fold_motif_information.append([[location, location + len(motif)], motif, intensity])

            if len(adjusted_motif_locations) >= 1:
                custom_print("Cleaved Motifs in the Sequence of Interest:")
                for i in range(len(adjusted_motif_locations)):
                    custom_print(f"Motif {i + 1}: {adjusted_motif_locations[i][1]}, Substrate Score: {adjusted_motif_locations[i][2] * 1: .2f}, Location: nucleotides {adjusted_motif_locations[i][0][0]}-{adjusted_motif_locations[i][0][1]}")
            else:
                custom_print("Cleaved Motifs in the Sequence of Interest: None")

            if cress_bool:
                fold_motif_information = sorted(fold_motif_information, key=lambda x: x[2], reverse=True)
                fold_motif_information = fold_motif_information[:4]

                for i in fold_motif_information:
                    sequence_to_fold = sequence_to_search[i[0][0]-10:i[0][1]+10]
                    ss, mfe = RNA.fold(sequence_to_fold)
                    i.append(sequence_to_fold)
                    i.append(ss)
                    i.append(mfe)

                fold_motif_information = sorted(fold_motif_information,
                                                  key=lambda x: x[5])

                likely_candidate = fold_motif_information[0]

                fold_file = f"StemLoop{dropdown_menu_huh_tag.value}_{fasta_file[0]}.svg"

                RNA.svg_rna_plot(likely_candidate[3], likely_candidate[4], fold_file)

                out_statement = f"\nPredicted Origin of Replication: {likely_candidate[1]}"

                custom_print(out_statement)
                print(out_statement)

            # Plotting
            plot_file = None
            substrate_scores = [score for sequence, score in sorted_motifs]
            if len(substrate_scores) != 0:
                plt.figure(figsize=(10, 6))
                plt.hist(substrate_scores, bins=50, color='#6666FF', edgecolor='None', alpha=0.5)
                plt.title(f'Substrate Score Histogram for {dropdown_menu_huh_tag.value}', fontsize=28)
                plt.xlabel('Substrate Score', fontsize=20)
                plt.ylabel('Count', fontsize=20)
                plt.xticks(fontsize=15)
                plt.yticks(fontsize=15)
                plt.xlim(0.0, 1.0)
                plt.grid(alpha=0.75)

                # Save plot to file
                plot_file = f"histogram_{dropdown_menu_huh_tag.value}_GB:{fasta_file[0]}.png"
                plt.savefig(plot_file)

                print("\nAnalysis complete. Downloading files.\n")

                # Display plot
                plt.show()

            out_text_str = (f'Output_{dropdown_menu_huh_tag.value}_CleavageProfileSubstrate_GB:{fasta_file[0]}.txt')

            if cress_bool:
                out_list = [plot_file, out_text_str, fold_file]
            else:
                out_list = [plot_file, out_text_str]

            master_zip_file = "HUHgleFASTAFiles.zip"
            with zipfile.ZipFile(master_zip_file, 'w') as master_zip:
                for link in out_list:
                    master_zip.write(link)

            files.download(master_zip_file)

    # Bind the callback function to the button
    submit_button.on_click(on_submit_clicked)

# Function to upload files and create dropdown menus and the submit button upon successful upload
def upload_files_and_create_interaction_elements():
    uploaded_files = files.upload()

    # Check if files were uploaded and validate file extension
    if uploaded_files:
        for filename in uploaded_files.keys():
            if is_fasta_file(filename):
                print(f"\nUploaded file '{filename}' is a valid FASTA file.")
                create_interaction_elements()
                if uploaded_files:
                    # Process each uploaded file
                    for filename, content in uploaded_files.items():
                        return(read_fasta(filename))
            else:
                print(f"\nUploaded file '{filename}' is not a FASTA file. Please upload a .FASTA file.")

# Main execution
fasta_file = upload_files_and_create_interaction_elements()
